In [ ]:
!pip install --upgrade openai pandas openpyxl

import pandas as pd
import time
from openai import OpenAI
from google.colab import files
from io import BytesIO

# 🔐 Set up OpenAI client with your API key
client = OpenAI(api_key="....")  # Replace with your key

# 📂 Upload your Excel file
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
xls = pd.ExcelFile(file_name)

# 🔁 Define prompt builder
def create_prompt(user, bot):
    return f"""Does the following bot response indicate a conversation breakdown?\nUser: {user}\nBot: {bot}\nAnswer with Yes or No only."""

# 🧠 Detect breakdowns per sheet
updated_sheets = {}

for sheet in xls.sheet_names:
    df = xls.parse(sheet)
    print(f"🔍 Processing: {sheet} ({len(df)} rows)")

    if "User_Message" not in df.columns or "Bot_Response" not in df.columns:
        print(f"❌ Skipping '{sheet}' — missing required columns.")
        continue

    detections = []
    for idx, row in df.iterrows():
        user = str(row.get("User_Message", "")).strip()
        bot = str(row.get("Bot_Response", "")).strip()



        if not bot:
            # If there's no bot reply, that is likely a breakdown
            detections.append("Yes")
            continue


        prompt = f"""You are reviewing a chatbot conversation to detect if the chatbot caused a breakdown.

Definition:
A chatbot-side breakdown means the **bot's reply is vague, off-topic, incomplete, irrelevant, or wrong** based on the user's message.

Now review this conversation:
User: {user if user.strip() else '[no message]'}
Bot: {bot if bot.strip() else '[no response]'}


Did the chatbot cause a breakdown?
"""
        try:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant detecting chatbot breakdowns."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0
            )
            detections.append(response.choices[0].message.content.strip())
            time.sleep(1)
        except Exception as e:
            print(f"⚠️ Error at row {idx}: {e}")
            detections.append("Error")

    df["GPT_Breakdown_Detection"] = detections
    updated_sheets[sheet] = df

# 💾 Save to one Excel file
output_buffer = BytesIO()
with pd.ExcelWriter(output_buffer, engine='openpyxl') as writer:
    for sheet_name, data in updated_sheets.items():
        data.to_excel(writer, sheet_name=sheet_name, index=False)

output_buffer.seek(0)
with open("updated_breakdown_detection.xlsx", "wb") as f:
    f.write(output_buffer.read())

files.download("updated_breakdown_detection.xlsx")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 24.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


Saving DBDC3_paired_user_bot.xlsx to DBDC3_paired_user_bot.xlsx
🔍 Processing: Sheet1 (2089 rows)
⚠️ Error at row 961: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
⚠️ Error at row 962: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
⚠️ Error at row 963: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-cod

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/openai/_base_client.py", line 1014, in request
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Client error '429 Too Many Requests' for url 'https://api.openai.com/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-d2f5a7525822>", line 58, in <cell line: 0>
    response = client.chat.completions.create(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packag


KeyboardInterrupt



In [ ]:
# 📊 Evaluation of GPT predictions

from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix

# Combine all sheets into a single DataFrame
combined_df = pd.concat(updated_sheets.values(), ignore_index=True)

# Normalize predictions to 0 (No) and 1 (Yes), handle 'Error' as well
combined_df["Model_Prediction"] = combined_df["GPT_Breakdown_Detection"].map({"Yes": 1, "No": 0, "Error": 0}) # Map 'Error' to 0 or another appropriate value
combined_df["Label"] = combined_df["Label"].astype(int)

# Remove rows with NaN in either 'Label' or 'Model_Prediction'
combined_df = combined_df.dropna(subset=['Label', 'Model_Prediction'])

# Compute metrics
precision = precision_score(combined_df["Label"], combined_df["Model_Prediction"])
recall = recall_score(combined_df["Label"], combined_df["Model_Prediction"])
accuracy = accuracy_score(combined_df["Label"], combined_df["Model_Prediction"])
conf_matrix = confusion_matrix(combined_df["Label"], combined_df["Model_Prediction"])

# Display metrics
print(f"✅ Precision: {precision:.2f}")
print(f"✅ Recall: {recall:.2f}")
print(f"✅ Accuracy: {accuracy:.2%}")
print("✅ Confusion Matrix:")
print(conf_matrix)

✅ Precision: 0.91
✅ Recall: 0.88
✅ Accuracy: 96.32%
✅ Confusion Matrix:
[[110   2]
 [  3  21]]
